Make sure to replace 'path/to/credentials.json' with the actual path to your Google Sheets API credentials file.

This program uses the requests library to send HTTP requests to the website, the BeautifulSoup library to parse the HTML content and extract the required information, and the gspread library to access and modify the Google Sheet. The program defines two functions: scrape_website to scrape the website and extract the information, and write_to_google_sheet to write the extracted information to the Google Sheet. The extracted information is then written to the Google Sheet specified in the URL.

Please note that the website structure may change, requiring updates to the program. Additionally, ensure that you have enabled the Google Sheets API, created the necessary credentials, and have the correct permissions to access and modify the Google Sheet.

1. Install libraries

In [3]:
!pip install requests
!pip install beautifulsoup4
!pip install gspread
!pip install oauth2client

2. Import libraries

In [4]:
import requests
from bs4 import BeautifulSoup
import gspread
from oauth2client.service_account import ServiceAccountCredentials

3. Define the function to scrape the website and extract the information

In [54]:
def scrape_website(url):
    # Make an HTTP GET request to the website
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the response
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all the cards on the website
        cards = soup.find_all('div', class_='card')

        # Initialize lists to store the extracted information
        titles = []
        urls = []
        # founding_years = []
        # total_funding = []

        # Extract the URLs, founding year, and total funding from each card
        for card in cards:
            # Extract the title
            title = card.find('h2', class_='text-lg font-bold').text
            titles.append(title)
            
            # Extract the URL
            url = card.find('a', class_='false max-w-full overflow-hidden text-clip leading-5 text-purple-400 line-clamp-1').text
            urls.append(url)

            # # Extract the founding year
            # founding_year = card.find('span', class_='founding-year').text
            # founding_years.append(founding_year)

            # # Extract the total funding
            # funding = card.find('span', class_='total-funding').text
            # total_funding.append(funding)

        # Return the extracted information
        return titles, urls #, founding_years, total_funding

    else:
        # The request was not successful, return None
        return None


In [55]:
# print(titles, urls, founding_years, total_funding)

4. Define the function to write the extracted information to the Google Sheet:

In [56]:
def write_to_google_sheet(urls, titles): #, founding_years, total_funding):
    # Define the scope and credentials
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/robertford/_Projects/ScraperGPT/affable-bastion-324718-46b5767bba57.json', scope)

    # Authenticate using the credentials
    gc = gspread.authorize(credentials)

    # Access the Google Sheet
    spreadsheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1-DrchRkuG5j-gsa2kI1sbw3cfamq2me7KkTrWRgSv2o/edit#gid=0')

    # Select the first sheet
    sheet = spreadsheet.get_worksheet(0)

    # Clear the existing data in the sheet
    sheet.clear()

    # Write the headers to the sheet
    sheet.update_cell(1, 1, 'titles')
    sheet.update_cell(1, 2, 'urls')
    # sheet.update_cell(1, 3, 'founding_year')
    # sheet.update_cell(1, 4, 'total_funding')

    # Write the extracted information to the sheet
    for i in range(len(urls)):
        sheet.update_cell(i+2, 1, urls[i])
        sheet.update_cell(i+2, 2, titles[i])
        # sheet.update_cell(i+2, 3, founding_years[i])
        # sheet.update_cell(i+2, 4, total_funding[i])


5. Call the function to scrape the website and write the extracted information to the Google Sheet

In [57]:
website_url = 'https://mad.firstmark.com/card'
# titles, urls, founding_years, total_funding = scrape_website(website_url)
titles, urls = scrape_website(website_url)
# write_to_google_sheet(titles, urls, founding_years, total_funding)
write_to_google_sheet(titles, urls)